In [ ]:
import datetime
import glob

import pandas as pd

from adhoc_energy_analytics.constants import RAW_DATA_DIR

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 200)

datetime_cols = [
    "UTC Timestamp (Interval Ending)",
    "Local Timestamp Central Time (Interval Beginning)",
    "Local Timestamp Central Time (Interval Ending)",
    "Local Date",
]

# Get a list of all files matching the pattern
df_file_list = glob.glob(f"{RAW_DATA_DIR}/spp_lmp_da_hr_interfaces_*.csv")

# Read all files into a single dataframe
df_list = [
    pd.read_csv(file, skiprows=3, parse_dates=datetime_cols) for file in df_file_list
]
df = pd.concat(df_list, ignore_index=True)

# Trim spaces from column names
df.columns = df.columns.str.strip()

# Set index to datetime
df.set_index("Local Timestamp Central Time (Interval Beginning)", inplace=True)

# Drop index duplicates (daylight savings)
df = df[~df.index.duplicated(keep="first")]

# Create a new index starting from the minimum date and ending at the maximum date
new_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq="h")

# Reindex the dataframe
df = df.reindex(new_index)

# Remove all dates where there is no local data for all 24 hours
null_hourly_value_counts = (
    df[df["Lamar LMP"].isna()].index.normalize().value_counts(sort=False)
)
null_dates = null_hourly_value_counts[null_hourly_value_counts == 24].index.to_numpy()
df = df[~df.index.normalize().isin(null_dates)]

# # Fill missing values with linear interpolation
# columns_to_fill = [
# ]
# df[columns_to_fill] = df[columns_to_fill].interpolate(
#     method="linear", limit_direction="both"
# )

print(len(df))
df.isna()["Lamar LMP"].sum()

36816


np.int64(5)

In [22]:
df[["Lamar LMP", "Lamar (Congestion)", "Lamar (Energy)", "Lamar (Loss)"]].corr().round(2)

,Lamar LMP,Lamar (Congestion),Lamar (Energy),Lamar (Loss)
Lamar LMP,1.00,0.19,1.00,-0.83
Lamar (Congestion),0.19,1.00,0.11,-0.03
Lamar (Energy),1.00,0.11,1.00,-0.85
Lamar (Loss),-0.83,-0.03,-0.85,1.00
